In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymysql

In [2]:
cnx = pymysql.connect(user='root', \
      password='cochin12', \
      host= '127.0.0.1', \
      port=3306, \
      db='world', \
      autocommit=True)

In [3]:
# Fetching the country, city and countrylanguage datasets
df_country = pd.read_sql_query('select * from Country', con=cnx,index_col='Code')
df_city = pd.read_sql_query('select * from city', con=cnx,index_col='ID')
df_countrylanguage = pd.read_sql_query('select * from countrylanguage', con=cnx,index_col=['CountryCode','Language'])

In [10]:
#Ques1
# This lists the top 10 countries by population size (with population > 50000000) in desc order of their population
# Note the population > 50000000 check is unnnecessary but we have still kept it to be consistent with the provided query
df_country.query("Population > 50000000").sort_values(by=['Population'], ascending=False).head(10)

,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
Code,,,,,,,,,,,,,,
CHN,China,Asia,Eastern Asia,9572900.0,-1523.0,1277558000,71.4,982268.0,917719.0,Zhongquo,People'sRepublic,Jiang Zemin,1891.0,CN
IND,India,Asia,Southern and Central Asia,3287263.0,1947.0,1013662000,62.5,447114.0,430572.0,Bharat/India,Federal Republic,Kocheril Raman Narayanan,1109.0,IN
USA,United States,North America,North America,9363520.0,1776.0,278357000,77.1,8510700.0,8110900.0,United States,Federal Republic,George W. Bush,3813.0,US
IDN,Indonesia,Asia,Southeast Asia,1904569.0,1945.0,212107000,68.0,84982.0,215002.0,Indonesia,Republic,Abdurrahman Wahid,939.0,ID
BRA,Brazil,South America,South America,8547403.0,1822.0,170115000,62.9,776739.0,804108.0,Brasil,Federal Republic,Fernando Henrique Cardoso,211.0,BR
PAK,Pakistan,Asia,Southern and Central Asia,796095.0,1947.0,156483000,61.1,61289.0,58549.0,Pakistan,Republic,Mohammad Rafiq Tarar,2831.0,PK
RUS,Russian Federation,Europe,Eastern Europe,17075400.0,1991.0,146934000,67.2,276608.0,442989.0,Rossija,Federal Republic,Vladimir Putin,3580.0,RU
BGD,Bangladesh,Asia,Southern and Central Asia,143998.0,1971.0,129155000,60.2,32852.0,31966.0,Bangladesh,Republic,Shahabuddin Ahmad,150.0,BD
JPN,Japan,Asia,Eastern Asia,377829.0,-660.0,126714000,80.7,3787042.0,4192638.0,Nihon/Nippon,Constitutional Monarchy,Akihito,1532.0,JP


In [5]:
#Ques2
# This lists the continents in alphabetical order along with the number of countries and total population in that continent
result1 = df_country.query("Population > 0 ").groupby(['Continent']).Name.count()
result2 = df_country.query("Population > 0 ").groupby(['Continent']).Population.sum()
result = pd.concat([result1, result2], axis=1, join='inner')
result.columns = ['Number_Countries', 'Population']
print(result)

               Number_Countries  Population
Continent                                  
Africa                       57   784475000
Asia                         51  3705025700
Europe                       46   730074600
North America                37   482993000
Oceania                      27    30401150
South America                14   345780000


In [6]:
# Ques 3
# This lists top 10 cities of USA by population size in desc order of the population
city = df_city[df_city['CountryCode'] == 'USA']
city[['Name','Population']].sort_values(by = ['Population'], ascending = False).set_index('Name').head(10)

,Population
Name,
New York,8008278
Los Angeles,3694820
Chicago,2896016
Houston,1953631
Philadelphia,1517550
Phoenix,1321045
San Diego,1223400
Dallas,1188580
San Antonio,1144646


In [14]:
# Ques 4
''' This lists the top 10 languages with the most speakers in the countries where they are official 
in desc order of the number of speakers'''
df_countrylanguage = df_countrylanguage.reset_index().set_index('CountryCode')
OfficialLanguages = df_countrylanguage[df_countrylanguage['IsOfficial'] == 'T']
OfficalLangPop = df_country.join(OfficialLanguages, how='inner')

OfficalLangPop['OfficalLangCount'] = OfficalLangPop['Population'] * OfficalLangPop['Percentage'] / 100
OfficalLangPop = OfficalLangPop.sort_values(by=['OfficalLangCount'], ascending = False).reset_index().head(10)
OfficalLangPop = OfficalLangPop[['Name', 'Language', 'OfficalLangCount']].set_index('Name')
print(OfficalLangPop)

                      Language  OfficalLangCount
Name                                            
China                  Chinese      1.175353e+09
India                    Hindi      4.044511e+08
United States          English      2.399437e+08
Brazil              Portuguese      1.658621e+08
Russian Federation     Russian      1.272448e+08
Bangladesh             Bengali      1.261844e+08
Japan                 Japanese      1.255736e+08
Mexico                 Spanish      9.106940e+07
Germany                 German      7.501637e+07
Vietnam             Vietnamese      6.929418e+07


In [12]:
# Ques 5
# This lists the top 5 languages by total number of speakers all over the world in desc order of the number of speakers
LangSpeakers = df_countrylanguage.join(df_country, how='inner').reset_index()
LangSpeakers['TotalCount'] = LangSpeakers['Population'] * LangSpeakers['Percentage'] / 100
LangSpeakers.groupby('Language').TotalCount.sum().reset_index().sort_values(by='TotalCount', ascending = False).head(5).set_index('Language')

,TotalCount
Language,
Chinese,1.191844e+09
Hindi,4.056331e+08
Spanish,3.550295e+08
English,3.470779e+08
Arabic,2.338392e+08
